# Module 13 — Deployment & Interactive Demos (Gradio, Flask, TFLite) — Expanded\n\nGenerated expanded notebook.

# Module 13 — Deployment & Interactive Demos (Expanded)

This notebook shows how to export models and build simple Gradio and Flask demos, plus convert a Keras model to TFLite for edge inference.

## 1 — Setup (install packages)

In [ ]:
!pip -q install -U gradio flask tensorflow --quiet
import os, numpy as np, tensorflow as tf
import matplotlib.pyplot as plt
print('TF', tf.__version__)


## 2 — Create or load a small Keras model (MNIST quick)

In [ ]:
from tensorflow.keras import layers, models
# small model for demo
mnist_model = models.Sequential([layers.Input((28,28,1)), layers.Conv2D(16,3,activation='relu'), layers.MaxPooling2D(), layers.Flatten(), layers.Dense(10, activation='softmax')])
mnist_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# create random weights (or train quickly)
import numpy as np
x_dummy = np.random.rand(100,28,28,1).astype('float32')
y_dummy = np.random.randint(0,10,size=(100,))
mnist_model.fit(x_dummy, y_dummy, epochs=1, batch_size=16, verbose=0)
mnist_model.save('/mnt/data/demo_mnist_model.h5')
print('Saved demo model to /mnt/data/demo_mnist_model.h5')

## 3 — Gradio demo (local)

In [ ]:
import gradio as gr
from PIL import Image
import numpy as np
# load model
model = tf.keras.models.load_model('/mnt/data/demo_mnist_model.h5')

def predict_fn(img: Image):
    img = img.convert('L').resize((28,28))
    arr = np.array(img).astype('float32')/255.0
    arr = arr.reshape(1,28,28,1)
    preds = model.predict(arr)
    label = int(preds.argmax())
    return label

demo = gr.Interface(fn=predict_fn, inputs=gr.Image(type='pil'), outputs='label')
# NOTE: demo.launch() will open a local server; in Colab set share=True
# demo.launch(share=True)
print('Gradio demo ready. In Colab, run demo.launch(share=True) to get a public link.')

## 4 — Minimal Flask inference endpoint (example)

In [ ]:
# This Flask app is for local deployment (not run in hosted notebook UI)
from flask import Flask, request, jsonify
app = Flask('demo_app')
model_flask = tf.keras.models.load_model('/mnt/data/demo_mnist_model.h5')

@app.route('/predict', methods=['POST'])
def predict():
    # expects JSON with 'image' as base64 or other scheme; here we return dummy
    return jsonify({'label': int(0), 'confidence': float(1.0)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

print('Flask example included. To run the Flask server, run this cell as a script in a terminal.')

## 5 — Convert to TFLite (Keras -> TFLite)

In [ ]:
# Convert demo Keras model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(mnist_model)
tflite_model = converter.convert()
tflite_path = '/mnt/data/demo_mnist_model.tflite'
open(tflite_path, 'wb').write(tflite_model)
print('Saved TFLite model to', tflite_path)


## 6 — Quick tips for classroom: hosting, sharing, and demos

- Use Gradio `share=True` in Colab to get a temporary public link for demos.
- Pre-run heavy training and save model weights; in class load weights and show inference quickly.
- For production, containerize Flask app and expose via a cloud VM or serverless function.
- For edge demos, use TFLite and run inference on device (Raspberry Pi, Android).